This notebook takes melspectrogram and tempogram as inputs and uses Resnet model . The model overfitted. Train to avoid overfitting. [Work in Progress]

In [1]:
# Resnet 

from __future__ import print_function, division
import os
import torch
import pandas as pd
import skimage
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage.transform import resize
import torch.nn as nn

In [2]:
## Load Data
labels = pd.read_csv('./input_label.csv')
#class MelspecDataset(Dataset):
class Dataset(Dataset):
    def __init__(self, csv_file, root_dir, tempo_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.tempo_dir = tempo_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.labels.iloc[idx, 0]+'.png')
        image = io.imread(img_name)
        
        l = self.labels.iloc[idx, 1:]       
        
        tempo_img_name = os.path.join(self.tempo_dir, self.labels.iloc[idx, 0]+'.png')
        tempo_image = io.imread(tempo_img_name)
        
        sample = {'image': image, 'label': l, 'tempo_image': tempo_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label, tempo_image = sample['image'], sample['label'], sample['tempo_image']
        

        # The input is a pandas Series
        if isinstance(label, pd.core.series.Series):
            label = label.values[0]
            
        label_id = ["Happy", "Angry", "Tender", "Scary", "Sad", "Funny"].index(label)
        
        #print (label, type(label))
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        
        #TODO: Convert image to RGB
        from PIL import Image
        im = Image.fromarray(image)
        image = np.array(im.convert('RGB'))
        image = image.transpose((2, 0, 1))
        #print (type(image), image.shape)
        image = resize(image, (3, 224, 224))
        #print("After resize",type(image), image.shape)
        

        
        # tempogram 
        im = Image.fromarray(tempo_image)
        tempo_image = np.array(im.convert('RGB'))
        tempo_image = tempo_image.transpose((2, 0, 1))
        tempo_image = resize(tempo_image, (3, 224, 224))
        
        return torch.from_numpy(image), torch.from_numpy(tempo_image), torch.from_numpy(np.array([label_id]))
        
transformed_dataset = Dataset(csv_file='./input_label.csv',
                              root_dir='./MelSpectrogram/',
                              tempo_dir='./tempogram/',
                              transform=ToTensor())

In [3]:
valid_size = 0.2
num_train = len(transformed_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))

In [4]:
# Shuffle the indices
np.random.seed(123)
np.random.shuffle(indices)

# params
batch_size = 16
num_workers = 1

train_idx, valid_idx = indices[split:], indices[:split]

from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


train_loader = torch.utils.data.DataLoader(
        transformed_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers)

valid_loader = torch.utils.data.DataLoader(
        transformed_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers)

In [5]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 25
num_classes = 6
batch_size = 64
learning_rate = 0.001

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
#https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

import torch.nn as nn
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(0.5)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dropout2 = nn.Dropout(0.5)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.dpo1 = nn.Dropout(0.5)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dpo2 = nn.Droput(0.5)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.dpo3 = nn.Dropout(0.5)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.dpo1(0.5)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.dpo2(0.5)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = self.dpo3(0.5)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc_inp1 = nn.Linear(512 * block.expansion, 256)  
        
        # 2nd input - replicate all the layers needed for the Resnet architecture
        self.conv1_inp2 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1_inp2 = nn.BatchNorm2d(64)
        self.relu_inp2 = nn.ReLU(inplace=True)
        self.maxpool_inp2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.dropout_inp2 = nn.Dropout(0.5)
        self.layer1_inp2 = self._make_layer(block, 64, layers[0])
        self.layer2_inp2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3_inp2 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4_inp2 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool_inp2 = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc_inp2 = nn.Linear(512 * block.expansion, 256)
        
        self.fc_out = nn.Linear(256 + 256, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        inp1 = x[0]
        inp2 = x[1]
        
        x = self.conv1(inp1)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc_inp1(x)
        
        inp2 = self.conv1(inp2)
        inp2 = self.bn1(inp2)
        inp2 = self.relu(inp2)
        inp2 = self.dropout(inp2)
        inp2 = self.maxpool(inp2)

        inp2 = self.layer1(inp2)
        inp2 = self.layer2(inp2)
        inp2 = self.layer3(inp2)
        inp2 = self.layer4(inp2)

        inp2 = self.avgpool(inp2)
        inp2 = inp2.view(inp2.size(0), -1)
        
        inp2 = self.fc_inp2(inp2)
        
        class_out = self.fc_out(torch.cat((x, inp2), 1))
        return class_out


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model



def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


In [8]:
model = resnet18(num_classes=6)
model = model.to(device, dtype=torch.float)

In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 25
loss_tracker = []
valid_cfn_matrix = []

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (mels_images, tempo_images, labels) in enumerate(train_loader):

        mels_images = mels_images.float()
        tempo_images = tempo_images.float()
        
        mels_images = mels_images.to(device, dtype=torch.float)
        tempo_images = tempo_images.to(device, dtype=torch.float)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model([mels_images, tempo_images])
        
        loss = criterion(outputs, labels.squeeze())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _x, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (labels.squeeze() == predicted).sum().item()
        loss_tracker.append(loss.item())
        if (i+1) % 100 == 0:
            #print (correct, total)
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
          
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        correct = 0
        total = 0
        for i, (mels_images, tempo_images, labels) in enumerate(valid_loader):
            mels_images = mels_images.float()
            tempo_images = tempo_images.float()
            mels_images = mels_images.to(device)
            tempo_images = tempo_images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model([mels_images, tempo_images])

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (labels.squeeze() == predicted).sum().item()

            all_predictions.extend(list(predicted.cpu().numpy()))
            all_labels.extend(list(labels.squeeze().cpu().numpy()))

        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
        
        import sklearn
        from sklearn.metrics import confusion_matrix
        cf_matrix = sklearn.metrics.confusion_matrix(all_predictions, all_labels)
        valid_cfn_matrix.append(cf_matrix)
        print (cf_matrix)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch [1/25], Step [100/508], Loss: 1.5082
Epoch [1/25], Step [200/508], Loss: 1.3345
Epoch [1/25], Step [300/508], Loss: 1.2063
Epoch [1/25], Step [400/508], Loss: 0.7352
Epoch [1/25], Step [500/508], Loss: 0.9971


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Test Accuracy of the model on the test images: 58.994578610152786 %
[[ 92   3  59   1   9  53]
 [ 18 174  22   8   5  12]
 [ 95   7 491  34 180  64]
 [  5  13  25 241  12   5]
 [  5   1  91  32 129   0]
 [ 38   3  29   1   2  70]]


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


...

Test Accuracy of the model on the test images: 57.91030064070971 %
[[ 96   9  66   3   7  50]
 [ 10 166  14   9   5   7]
 [107  16 479  37 200  55]
 [  2   5  24 249  15   6]
 [  4   0 109  10 103   4]
 [ 34   5  25   9   7  82]]


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch [25/25], Step [100/508], Loss: 0.0658
Epoch [25/25], Step [200/508], Loss: 0.0556
Epoch [25/25], Step [300/508], Loss: 0.5709
Epoch [25/25], Step [400/508], Loss: 0.0634
Epoch [25/25], Step [500/508], Loss: 0.0413


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Test Accuracy of the model on the test images: 57.71315919172006 %
[[120   8 108   0  16  58]
 [  6 152   6   4   2   3]
 [ 84  23 444  33 177  57]
 [  2  12  29 264  25   4]
 [  9   1 107  12 110   1]
 [ 32   5  23   4   7  81]]


In [13]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
all_predictions = []
all_labels = []
with torch.no_grad():
    correct = 0
    total = 0
    for i, (mels_images, tempo_images, labels) in enumerate(valid_loader):
        mels_images = mels_images.float()
        tempo_images = tempo_images.float()
        
        mels_images = mels_images.to(device)
        tempo_images = tempo_images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model([mels_images, tempo_images])
 
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (labels.squeeze() == predicted).sum().item()

        all_predictions.extend(list(predicted.cpu().numpy()))
        all_labels.extend(list(labels.squeeze().cpu().numpy()))

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Test Accuracy of the model on the test images: 48.74322326269098 %


In [14]:
import sklearn
from sklearn.metrics import confusion_matrix
print (sklearn.metrics.confusion_matrix(all_predictions, all_labels))

[[146  39 158   7  44  72]
 [  2  80   1   3   1   0]
 [ 51  21 381  77 159  43]
 [  4   6  13 176   5   0]
 [  5   1 119  43 122   5]
 [ 45  54  45  11   6  84]]


``` Previous run 
Test Accuracy of the model on the test images: 60.177427304090685 %
[[ 94   2  68   0   5  41]
 [ 13 178  23  20   4  14]
 [ 83   7 540  36 244  50]
 [  1   5  21 249  18   0]
 [  3   0  33   9  61   0]
 [ 59   9  32   3   5  99]]
```

Note --

model.eval - acc =48.74%
with torch.no_grad(): - acc = 62.25%